In [38]:
import imp
import Data
import utils
import Predictors, Road_Graph, Neural_Network
imp.reload(Data)
imp.reload(Neural_Network)
imp.reload(utils)
imp.reload(Predictors)
imp.reload(Road_Graph)

from Road_Graph import *
from Predictors import *
from Data import *
from utils import *
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
import torch
from torch import nn, autograd
import torch.nn.functional as F
import pickle

In this part, we focus on a simple road map (rep as a graph below), and generate random path on it.
<img src="../img/naive_road.png" alt="Drawing" style="width: 400px;"/>

In [3]:
graph_dict = {}
graph_dict = graph_dict.fromkeys(range(24))
edges = [(i, i+1) for i in range(5)] \
        + [(i+6, i+7) for i in range(5)]\
                        +[(i+12, i+13) for i in range(5)]\
                     + [(i+18, i+19) for i in range(5)]\
                     + [(0,6),(6,12), (12,18)]\
                     + [(pair[0]+1, pair[1]+1) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+2, pair[1]+2) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+3, pair[1]+3) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+4, pair[1]+4) for pair in [(0,6),(6,12), (12,18)]]\
                     + [(pair[0]+5, pair[1]+5) for pair in [(0,6),(6,12), (12,18)]]
for i,j in edges:
    if not graph_dict[i]:
        graph_dict[i] = [j]
    else:
        graph_dict[i].append(j)
    if not graph_dict[j]:
        graph_dict[j] = [i]
    else:
        graph_dict[j].append(i)

graph = Road_Graph(graph_dict)

# Markov Data

In [ ]:
def path_to_training_pair2(path):
    return path[1:], path[:-1]

def get_prob(x):
    nb = graph.neighbors(x)
    W_list = [W[j].view(-1,1) for j in nb]
    Ws = torch.cat(W_list, dim = 1)
    W_next = model(W[x])
    #print('W_next:', W_next)
    #print(W_next.size(), Ws.size())
    W_next_rep = W_next.view(-1,1).repeat(1,Ws.size()[1])
    #print(W_next_rep.size(), Ws.size())
    logit = torch.sum(W_next_rep*Ws, 0)
    prob = F.softmax(logit, dim = 0)
    return prob

def get_loss(x, y):
    nb = graph.neighbors(x)
    #import pdb; pdb.set_trace()
    W_list = [W[j].view(-1,1) for j in nb]
    Ws = torch.cat(W_list, dim = 1)
    W_next = model(W[x])
    #print('W_next:', W_next)
    #print(W_next.size(), Ws.size())
    W_next_rep = W_next.view(-1,1).repeat(1,Ws.size()[1])
    #print(W_next_rep.size(), Ws.size())
    logit = torch.sum(W_next_rep*Ws, 0)
    prob = F.softmax(logit, dim = 0)
    loss = -torch.log(prob[np.where(nb == y)])
    return loss

In [ ]:
dtype = torch.FloatTensor

data = random_walk_data(graph,10000)

im_D = 5
N = 24
h_D = 100


loss_list = []
W = Variable(torch.randn(N,im_D).type(dtype), requires_grad=True)
model = torch.nn.Sequential(
        torch.nn.Linear(im_D, h_D),
        torch.nn.ReLU(),
        torch.nn.Linear(h_D, im_D),
    )
for k in range(len(data)):
    path = data[k]
    x,y = path_to_training_pair2(path)
    
    optimizer = torch.optim.Adam(list(model.parameters()) + [W], lr=0.001) # remove [W] for a fixed random embeding
    for i in range(len(x)):
        loss = get_loss(x[i], y[i])
        loss_list.append(loss)
        
    if k%50 == 0:
        batch_loss = torch.sum(torch.cat(loss_list)) / len(loss_list)
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        loss_list = []
        print(batch_loss.data.numpy())
 

In [ ]:
p = np.array([1/3,1/3,1/3])
-np.sum(p * np.log(p))

In [ ]:
def opt_log_loss():
    s = 0
    for i in range(24):
        p_ = get_prob(i).data.numpy()
        n = len(get_prob(i))
        s += -np.sum(np.ones(n) /n * np.log(np.ones(n)/n))
    return s/24
opt_log_loss()

In [ ]:
def log_loss():
    s = 0
    for i in range(24):
        p_ = get_prob(i).data.numpy()
        n = len(get_prob(i))
        s += -np.sum(np.ones(n) /n * np.log(p_))
    return s/24
log_loss()

In [ ]:
def log_loss():
    s = 0
    for i in range(24):
        p_ = get_prob(i).data.numpy()
        n = len(get_prob(i))
        s += -np.sum(np.ones(n) /n * np.log(p_))
    return s/24
log_loss()

In [ ]:
get_prob(1)

# Hist Dependent Data

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.gru = nn.GRU(input_size, hidden_size, n_layers)
        self.lin = nn.Linear(hidden_size, output_size)
    
    def forward(self, inp, hidden):
        output, hidden = self.gru(inp.view(1, 1, self.input_size), hidden)
        output = self.lin(output) 
        return output, hidden        

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [ ]:
def path_loss(path):
    h = model.init_hidden()
    output_list = []
    for i in path:
        o, h = model(W[i], h)
        output_list.append(o)
    loss_list = []
    for i in range(len(path) - 1):
        x = path[i]
        y = path[i+1]
        nb = graph.neighbors(x)
        W_list = [W[j].view(-1,1) for j in nb]
        Ws = torch.cat(W_list, dim = 1)
        W_next = output_list[i]
        W_next_rep = W_next.view(-1,1).repeat(1,Ws.size()[1])
        logit = torch.sum(W_next_rep*Ws, 0)
        prob = F.softmax(logit, dim = 0)
        loss_list.append(-torch.log(prob[np.where(nb == y)]))
    return torch.sum(torch.cat(loss_list))

def get_prob(path):
    h = model.init_hidden()
    for i in path:
        o, h = model(W[i], h)
    x = path[-1]
    nb = graph.neighbors(x)
    W_list = [W[j].view(-1,1) for j in nb]
    Ws = torch.cat(W_list, dim = 1)
    W_next = o
    W_next_rep = W_next.view(-1,1).repeat(1,Ws.size()[1])
    logit = torch.sum(W_next_rep*Ws, 0)
    prob = F.softmax(logit, dim = 0)
    return prob

In [ ]:
data = random_walk_data(graph, 10000, go_back=False)

In [ ]:
im_D = 50
N = 24
h_D = 100
dtype = torch.FloatTensor


W = Variable(torch.randn(N,im_D).type(dtype), requires_grad=True)
model = RNN(im_D, h_D, im_D)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # remove [W] for a fixed random embeding

loss_list = []
s = 0
for k in range(len(data)):
    path = data[k]
    loss = path_loss(path)
    loss_list.append(loss)    
    s += len(path)
    if k%50 == 0:
        batch_loss = torch.sum(torch.cat(loss_list)) / s
        s = 0
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        loss_list = []
        print(batch_loss.data.numpy())
 

In [ ]:
get_prob([1,2,3,4])

# Modulized Code

In [10]:
data = random_walk_data(graph, 10000, go_back=False)

In [39]:
rnn_pd = RNN_Predictor(graph)

In [ ]:
rnn_pd.train(data)

[ 1.43386555]
[ 1.28774869]
[ 1.34153068]
[ 1.27314484]
[ 1.23466599]
[ 1.19762063]
[ 1.17873383]
[ 1.15821397]
[ 1.16038024]
[ 1.04382944]
[ 1.09409392]
[ 1.04834044]
[ 1.07298815]
[ 1.04028487]
[ 0.95232373]
[ 0.98204458]
[ 0.97956848]
[ 0.95116878]
[ 0.94344908]
[ 0.87796742]
[ 0.95411146]
[ 0.85435915]
[ 0.89130545]
[ 0.87831348]
[ 0.89541167]
[ 0.89544797]
[ 0.89993894]
[ 0.88904434]
[ 0.86201966]
[ 0.85485506]
[ 0.89148772]
[ 0.87167549]
[ 0.86684632]
[ 0.8418957]
[ 0.86877811]
[ 0.84373087]
[ 0.817586]
[ 0.83476841]
[ 0.88498414]
[ 0.82680124]
[ 0.89422202]
[ 0.82049465]
[ 0.82078958]
[ 0.83400857]
[ 0.87270308]
[ 0.83814883]
[ 0.82032931]
[ 0.84316963]
[ 0.79578835]
[ 0.83615142]
[ 0.87918681]
[ 0.79129088]
[ 0.82373762]
[ 0.84966844]
[ 0.8332417]
[ 0.8528778]
[ 0.83399636]
[ 0.8213982]
[ 0.86966777]
[ 0.81227708]
[ 0.83005953]
[ 0.83015913]
[ 0.80657768]
[ 0.8018437]
[ 0.79199207]
[ 0.77938694]
[ 0.81173289]
[ 0.81711346]
[ 0.81508952]
[ 0.81835002]
[ 0.77864885]
[ 0.83912337]

In [ ]:
graph.graph.number_of_nodes()